In [2]:
import config
import numpy as np
import pandas as pd
import Test_without_Training

#if __name__ == "__main__":
trainer = Test_without_Training.TremorModelTrainer(config, subject="Hunmin")
X_train_H, y_train_H, X_test_H, y_test_H = trainer.return_K_data(K=28)

trainer = Test_without_Training.TremorModelTrainer(config, subject="Xianyu")
X_train_X, y_train_X, X_test_X, y_test_X = trainer.return_K_data(K=6)

X_train = np.concatenate((X_train_H, X_train_X), axis=0)
y_train = np.concatenate((y_train_H, y_train_X), axis=0)

indices = np.arange(X_train.shape[0])
np.random.shuffle(indices)

X_train, y_train = X_train[indices], y_train[indices]

acc_H, Model_H_X = trainer.train_multiple_dataset(X_train, y_train, X_test_H, y_test_H)  #Mixed
acc_H2, Model_X = trainer.train_multiple_dataset(X_train_X, y_train_X, X_test_H, y_test_H)  #Only X
acc_H3, Model_H = trainer.train_multiple_dataset(X_train_H, y_train_H, X_test_H, y_test_H)  #Only H

loss, acc_X = Model_H_X.evaluate(X_test_X, y_test_X, verbose=0)
loss, acc_X2 = Model_X.evaluate(X_test_X, y_test_X, verbose=0)
loss, acc_X3 = Model_H.evaluate(X_test_X, y_test_X, verbose=0)

Dataset 1/34 - Session Exp_2025-05-27/E8331D05289A/
Dataset 2/34 - Session Exp_2025-06-18/E9AD0E7DCC2B/
Dataset 3/34 - Session Exp_2025-06-20-v1/E9AD0E7DCC2B/
Dataset 4/34 - Session Exp_2025-06-20-v2/E9AD0E7DCC2B/
Dataset 5/34 - Session Exp_2025-06-20-v3/E9AD0E7DCC2B/
Dataset 6/34 - Session Exp_2025-06-20-v4/E9AD0E7DCC2B/
Dataset 7/34 - Session Exp_2025-06-20-v5/E9AD0E7DCC2B/
Dataset 8/34 - Session Exp_2025-06-20-v6/E9AD0E7DCC2B/
Dataset 9/34 - Session Exp_2025-06-20-v7/E9AD0E7DCC2B/
Dataset 10/34 - Session Exp_2025-06-20-v8/E9AD0E7DCC2B/
Dataset 11/34 - Session Exp_2025-06-23-v1/E9AD0E7DCC2B/
Dataset 12/34 - Session Exp_2025-06-23-v2/E9AD0E7DCC2B/
Dataset 13/34 - Session Exp_2025-06-23-v3/E9AD0E7DCC2B/
Dataset 14/34 - Session Exp_2025-06-23-v4/E9AD0E7DCC2B/
Dataset 15/34 - Session Exp_2025-06-24-v1/E9AD0E7DCC2B/
Dataset 16/34 - Session Exp_2025-06-24-v2/E9AD0E7DCC2B/
Dataset 17/34 - Session Exp_2025-06-24-v3/E9AD0E7DCC2B/
Dataset 18/34 - Session Exp_2025-06-24-v4/E9AD0E7DCC2B/
Dataset

In [3]:
print(f"Trained with Subject H_X / Test with Subject H: {acc_H:.2f}%")
print(f"Trained with Subject X       / Test with Subject H: {acc_H2:.2f}%")
print(f"Trained with Subject H       / Test with Subject H: {acc_H3:.2f}%")
print("\n")
print(f"Trained with Subject H and X / Test with Subject X: {acc_X*100:.2f}%")
print(f"Trained with Subject X       / Test with Subject X: {acc_X2*100:.2f}%")
print(f"Trained with Subject H       / Test with Subject X: {acc_X3*100:.2f}%")

Trained with Subject H_X / Test with Subject H: 90.56%
Trained with Subject X       / Test with Subject H: 76.75%
Trained with Subject H       / Test with Subject H: 91.67%


Trained with Subject H and X / Test with Subject X: 67.95%
Trained with Subject X       / Test with Subject X: 71.67%
Trained with Subject H       / Test with Subject X: 53.77%
